# Object Recognition Task using MLP and CNN on CIFAR-10 Dataset
Dans ce TP nous allons faire de la classification multi-classes de 10 types d'objets.

### Importation des modules

In [ ]:
import numpy as np

import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras import datasets, layers, models
from keras.utils.np_utils import to_categorical 
from keras import regularizers
from keras.layers import Dense, Activation, Dropout, BatchNormalization
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.optimizers import SGD
import tensorflow.python.platform.build_info as build

In [ ]:
print(tf.__version__)

# 1. Chargement et analyse du dataset CIFAR-10 (depuis les jeux de données Keras)

### Chargement du jeu de données CIFAR-10 et division en jeu d'entrainement et de test

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

### Analyse du jeu de données

### <font color="red">**Exo1**</font> :  
- Combien d'images contient le jeu de données d'entrainement? et le jeu de test?  

- Quelle est la dimension des images d'entrée et le nombre de canaux ?  

- Quelle est la dimension du tenseur d'entrée ?  


In [ ]:
# Réponses: 
print(_____.____)
print(_____.____)
print(_____.____)
print(_____.____)

### <font color="red">**Exo2**</font> :  
- Combien y a-t-il de classes de sortie ?  

- Quelles sont les valeurs des labels ?  


In [ ]:
# Réponses: 
print(_____(train_labels))
print(_____(test_labels))

In [ ]:
# Creation d'une liste de tous les labels pour les classes
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

### Affichage de quelques images du jeu de données d'entrainement (train_images) 
On affiche le label de la classe correspondante (en utilisant class_names défini précédemment)

In [ ]:
plt.figure(figsize=[10,10])
for i in range (25):    # for first 25 images
  plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(train_images[i], cmap=plt.cm.binary)
  plt.xlabel(class_names[train_labels[i][0]])

plt.show()

### <font color="red">**Exo3**</font> :  
- Déterminer le nombre d'observations par classe dans le jeu d'entrainement et de test.  

- Est-ce un jeu de données équilibré ?  



In [ ]:
# Réponse:
train_labels_count = np.unique(______, return_counts=True)
for i in range(len(train_labels_count[0])):
    print(f"Label {i} : {_____} ({_____/_____*100:.2f}%)")

In [ ]:
# Réponse:
test_labels_count = np.unique(______, return_counts=True)
for i in range(len(test_labels_count[0])):
    print(f"Label {i} : {_____} ({_____/_____*100:.2f}%)")

# Data Preprocessing
- Comme nous allons utiliser un MLP, il est nécessaire de changer la forme (`reshape`) du tenseur d'entrée (de la forme **(50000, 32, 32, 3)**) en un tenseur de la forme **(50000, 3072)**
- **Normalisation** des valeurs de pixel entre **[0-1]**
- Les labels de sortie sont mis sous la forme `categorical` car nous allons utiliser une loss 'categorical_crossentropy'


### <font color="red">**Exo4**</font> : Compléter le code de la cellule suivante afin
- d'adapter la forme des données d'entrainement et de test au format attendu par le MLP
- de normaliser les données d'entrée (normalisation min-max)

In [ ]:
# Reshape du tenseur d'entrée
X_train = np.reshape(train_images,(_____,_____))
X_test = np.reshape(test_images,(_____,_____))

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalisation
X_train = X_train / _____
X_test = X_test / _____

# Comme nous allons utiliser une loss 'categorical_crossentropy', nous devons avoir les labels de sortie sous la forme 'categorical' 
num_classes = 10
Y_train = to_categorical(_____, num_classes)
Y_test = to_categorical(_____, num_classes)

# 3. Entrainement d'un Multi-Layer Perceptron (MLP)

### <font color="red">**Exo5**</font> : Construire un modèle MLP comportant les couches suivantes  
+ 1 couche **Dense** de 256 neurones avec activation 'relu'
+ 1 couche **Dense** de 256 neurones avec activation 'relu'
+ 1 couche **Dense** de sortie de 10 neurones avec activation 'softmax'

In [ ]:
# Réponse :
MLP_model = Sequential()
MLP_model.add(_____)
...

### Compiling the MLP model

In [ ]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
MLP_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
MLP_model.summary()

### Training the MLP
- Batch Size : 64
- Nb epochs : 20

In [ ]:
history = MLP_model.fit(X_train, Y_train, epochs=20, batch_size=64, verbose=1, validation_split=0.2)

### <font color="red">**Exo6**</font> : Sauvegarder le MLP entrainé

In [ ]:
MLP_model._____('MLP_CIFAR10_model.h5')

In [ ]:
#MLP_model = tf.keras.models.load_model('MLP_CIFAR10_model.h5')

### <font color="red">**Exo7**</font> : Afficher les courbes de loss et d'accuracy du MLP en fonction du nombre d'epoch. Que concluez-vous ?

In [ ]:
# Loss curve
...

In [ ]:
# Accuracy curve
...

### <font color="red">**Exo8**</font> : Evaluation du modèle MLP sur les données de test. Afficher l'accuracy obtenue

In [ ]:
score = MLP_model._____(_____, _____, batch_size=32, verbose=0)

In [ ]:
print(f"Le MLP a une accuracy de {score[1]*100:.2f}%")

### Afficher les métriques (precision, recall...) et la matrice de confusion du MLP (sur les données de test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# The predict function returns a 2-dimension array (10000, 10). 
# For each test input, it returns a probability for the 10 output classes
y_softmax = MLP_model.predict(X_test)

In [ ]:
# The argmax numpy function returns the class having the highest probability
y_pred = np.argmax(y_softmax, axis=-1)

### <font color="red">**Exo9**</font> : A l'aide la fonction `classification_report` afficher la précision, recall et f1-score pour le MLP. Conclure.

In [ ]:
print("EVALUATION ON TESTING DATA")
print(classification_report(_____, _____, target_names=class_names, digits=4))

### <font color="red">**Exo10**</font> : Afficher et analyser la matrice de confusion obtenue à partir du MLP. Quelles classes posent plus particulièrement problème ?

In [ ]:
label_map = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

In [ ]:
NUM_CLASS = 10
labels_to_display=[]
for i in range(NUM_CLASS):
    labels_to_display.append(i)
    
label_names = []
label_ticks = []
for key in label_map:
  label_names.append(label_map[key])
  label_ticks.append(key)
    
# Construction de la matrice de confusion
cm = confusion_matrix(_____, _____, normalize=None, labels=labels_to_display)

# Affichage de la matrice de confusion
disp = ConfusionMatrixDisplay(confusion_matrix = ______)
fig, ax = plt.subplots(figsize=(8, 8))
cmap = plt.get_cmap('Blues')
disp.plot(ax=ax, cmap=cmap, xticks_rotation="vertical")
plt.xticks(label_ticks[0:NUM_CLASS], label_names[0:NUM_CLASS], rotation='vertical')
plt.yticks(label_ticks[0:NUM_CLASS], label_names[0:NUM_CLASS], rotation='horizontal')
plt.show()

# 4. Entrainement d'un Réseau de Neurones Convolutif (CNN)

### Reload CIFAR10 dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

### Data Preprocessing

In [ ]:
# Pour le CNN, il n'est pas nécessaire de changer la forme des données d'entrainement et de test 
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalization of pixel values (to [0-1] range)
X_train = X_train / 255.0
X_test = X_test / 255.0

# One hot encoding the target class (labels)
num_classes = 10
Y_train = to_categorical(train_labels, num_classes)
Y_test = to_categorical(test_labels, num_classes)

### <font color="red">**Exo11**</font> : Construire un modèle CNN comportant les couches suivantes  
+ 1 couche de **Convolution 2D** avec 32 filtres de taille 3x3, padding='same' et activation 'relu'
+ 1 couche de **BatchNormalization**
+ 1 couche de **Convolution 2D** avec 32 filtres de taille 3x3, padding='same' et activation 'relu'
+ 1 couche de **BatchNormalization**
+ 1 couche de **MaxPooling 2D** de taille 2x2
+ 1 couche de **dropout** avec une probabilité de 0.3
---
+ 1 couche de **Convolution 2D** avec 64 filtres de taille 3x3, padding='same' et activation 'relu'
+ 1 couche de **BatchNormalization**
+ 1 couche de **Convolution 2D** avec 64 filtres de taille 3x3, padding='same' et activation 'relu'
+ 1 couche de **BatchNormalization**
+ 1 couche de **MaxPooling 2D** de taille 2x2
+ 1 couche de **dropout** avec une probabilité de 0.5
---
+ 1 couche de **Convolution 2D** avec 128 filtres de taille 3x3, padding='same' et activation 'relu'
+ 1 couche de **BatchNormalization**
+ 1 couche de **Convolution 2D** avec 128 filtres de taille 3x3, padding='same' et activation 'relu'
+ 1 couche de **BatchNormalization**
+ 1 couche de **MaxPooling 2D** de taille 2x2
+ 1 couche de **dropout** avec une probabilité de 0.5
---
+ 1 couche **Flatten**
+ 1 couche **Dense** de 128 neurones avec activation 'relu'
+ 1 couche de **BatchNormalization**
+ 1 couche de **dropout** avec une probabilité de 0.5
+ 1 couche **Dense** de sortie de 10 neurones avec activation 'softmax'

In [ ]:
# Creating a sequential model and adding layers to it
CNN_model = Sequential()

CNN_model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
CNN_model.add(layers._____)
CNN_model.add(layers._____)
...

In [ ]:
# Checking the model summary
CNN_model.summary()

## Compiling the Model
- Optimizer used during Back Propagation for weight and bias adjustment - Adam (adjusts the learning rate adaptively).
- Loss Function used - Categorical Crossentropy (used when multiple categories/classes are present).
- Metrics used for evaluation - Accuracy.


In [ ]:
CNN_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

## Fitting the Model
- Batch Size : 64
- Nb epochs : 20

In [ ]:
history = CNN_model.fit(X_train, Y_train, batch_size=64, epochs=20, verbose=1, validation_data=(X_test, Y_test))

### <font color="red">**Exo12**</font> : Sauvegarder le CNN entrainé

In [ ]:
CNN_model._____('CNN_CIFAR10_model.h5')

In [ ]:
# Recréé exactement le même model, incluant poids et optimizer.
# CNN_model = tf.keras.models.load_model('CNN_CIFAR10_model.h5')

### <font color="red">**Exo13**</font> : Afficher les courbes de loss et d'accuracy du CNN en fonction du nombre d'epoch. Que concluez-vous ?

In [ ]:
# Loss curve
...

In [ ]:
# Accuracy curve
...

### <font color="red">**Exo14**</font> : Evaluation du modèle CNN sur les données de test. Afficher l'accuracy obtenue

In [ ]:
score = CNN_model._____(_____, _____, batch_size=64, verbose=0)

In [ ]:
print(f"Le CNN a une accuracy de {score[1]*100:.2f}%")

### Afficher les métriques (precision, recall...) et la matrice de confusion du CNN (sur les données de test)

In [ ]:
# The predict function returns a 2-dimension array (10000, 10). 
# For each test input, it returns a probability for the 10 output classes
y_softmax = CNN_model.predict(X_test)

In [ ]:
# The argmax numpy function returns the class having the highest probability
y_pred = np.argmax(y_softmax, axis=-1)

### <font color="red">**Exo15**</font> : A l'aide la fonction `classification_report` afficher la précision, recall et f1-score pour le CNN. Conclure.

In [ ]:
print("EVALUATION ON TESTING DATA")
print(classification_report(_____, _____, target_names=class_names, digits=4))

### <font color="red">**Exo16**</font> : Afficher et analyser la matrice de confusion obtenue à partir du CNN. Quelles classes posent encore problème? 

In [ ]:
NUM_CLASS = 10
labels_to_display=[]
for i in range(NUM_CLASS):
    labels_to_display.append(i)
    
label_names = []
label_ticks = []
for key in label_map:
  label_names.append(label_map[key])
  label_ticks.append(key)
    
# Construction de la matrice de confusion
cm = confusion_matrix(_____, _____, normalize=None, labels=labels_to_display)

# Affichage de la matrice de confusion
disp = ConfusionMatrixDisplay(confusion_matrix = _____)
fig, ax = plt.subplots(figsize=(8, 8))
cmap = plt.get_cmap('Blues')
disp.plot(ax=ax, cmap=cmap, xticks_rotation="vertical")
plt.xticks(label_ticks[0:NUM_CLASS], label_names[0:NUM_CLASS], rotation='vertical')
plt.yticks(label_ticks[0:NUM_CLASS], label_names[0:NUM_CLASS], rotation='horizontal')
plt.show()

## Prédiction à partir d'images du jeu de test
- Prenons 25 images du jeu de test et voyons combien ont été correctment classées par le modèle CNN.

In [ ]:
# Making the Predictions
pred = CNN_model.predict(X_test)
print(pred)

# Converting the predictions into label index 
pred_classes = np.argmax(pred, axis=1)
print(pred_classes)

In [ ]:
# Plotting the Actual vs. Predicted results

fig, axes = plt.subplots(5, 5, figsize=(15,15))
axes = axes.ravel()

for i in np.arange(0, 25):
    axes[i].imshow(test_images[i])
    axes[i].set_title("True: %s \nPredict: %s" % (class_names[np.argmax(Y_test[i])], class_names[pred_classes[i]]))
    axes[i].axis('off')
    plt.subplots_adjust(wspace=1)